### Project 5: Relational Database
### Author: Chad Bell
### Date: 3-10-2023
### Description: Recreate the PREVAIL database

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import psycopg2

In [6]:
# Establish a connection with the postgres database
con = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="example_password")
# Create a cursor object
cur = con.cursor()

In [10]:
con.commit()
# con.rollback()

In [8]:

# Create loan types table
cur.execute("DROP TABLE IF EXISTS loan_types;")
cur.execute("""CREATE TABLE loan_types (
    id serial PRIMARY KEY NOT NULL,
    name TEXT,
    interest_rate NUMERIC,
    loan_term NUMERIC,
    pay_frequency NUMERIC,
    maturity_bonus NUMERIC,
    referral_rate NUMERIC,
    type_notes TEXT
    );"""
            )

# Create loans table
cur.execute("DROP TABLE IF EXISTS loans;")
cur.execute("""CREATE TABLE loans (
    id serial PRIMARY KEY NOT NULL,
    initial_start_date DATE,
    qualified BOOLEAN,
    ira_num TEXT,
    loan_notes TEXT,
    partner_paid DATE,
    promissory_note BYTEA,
    FOREIGN KEY (id) REFERENCES loan_types (id)
  );"""
            )
# Create loan instances table
cur.execute("DROP TABLE IF EXISTS loan_instances;")
cur.execute("""CREATE TABLE loan_instances (
    id serial PRIMARY KEY NOT NULL,
    amount NUMERIC,
    stage TEXT,
    app_submitted TIMESTAMP,
    funds_wired TIMESTAMP,
    start_date DATE,
    date_paid TIMESTAMP,
    loan_num NUMERIC,
    instance_notes TEXT,
    loan_id INTEGER REFERENCES loans (id)
  );"""
            )

# Create clients table
cur.execute("DROP TABLE IF EXISTS clients;")
cur.execute("""CREATE TABLE clients (
    id serial PRIMARY KEY NOT NULL,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT,
    address TEXT,
    city TEXT,
    state TEXT,
    zip TEXT,
    client_notes TEXT
    );"""
            )
# Create referring partners table
cur.execute("DROP TABLE IF EXISTS referring_partners;")
cur.execute("""CREATE TABLE referring_partners (
    id serial PRIMARY KEY NOT NULL,
    name TEXT,
    email TEXT,
    referral_rate NUMERIC,
    partner_notes TEXT
  );""")

# Many-to-many relationship between clients and referring partners
cur.execute("DROP TABLE IF EXISTS client_referring_partner;")
cur.execute("""CREATE TABLE client_referring_partner (
    client_id INTEGER REFERENCES clients (id),
    referring_partner_id INTEGER REFERENCES referring_partners (id),
    PRIMARY KEY (client_id, referring_partner_id)
  );"""
            )

# Many-to-many relationship between loans and clients
cur.execute("DROP TABLE IF EXISTS loan_client;")
cur.execute("""CREATE TABLE loan_client (
    loan_id INTEGER REFERENCES loans (id),
    client_id INTEGER REFERENCES clients (id),
    PRIMARY KEY (loan_id, client_id)
  );"""
            )

In [9]:
cur.execute("SELECT * FROM information_schema.tables WHERE table_schema='public';")
cur.fetchall()

[('postgres',
  'public',
  'loan_types',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'loans',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'loan_instances',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'clients',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'client_referring_partner',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'referring_partners',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None),
 ('postgres',
  'public',
  'loan_client',
  'BASE TABLE',
  None,
  None,
  None,
  None,
  None,
  'YES',
  'NO',
  None)]

In [ ]:
def create_loan_type(name, interest_rate, loan_term, pay_frequency, maturity_bonus, referral_rate, type_notes):
    """Create a new loan type"""
    cur.execute(f"""INSERT INTO loan_types (name, interest_rate, loan_term, pay_frequency, maturity_bonus, referral_rate, type_notes)
                VALUES ({name}, {interest_rate}, {loan_term}, {pay_frequency}, {maturity_bonus}, {referral_rate}, {type_notes});""")
    con.commit()

def create_loan(qualified, ira_num, loan_notes, partner_paid, promissory_note):
    """Create a new loan"""
    cur.execute(f"""INSERT INTO loans (qualified, ira_num, loan_notes, partner_paid, promissory_note)
                VALUES ({qualified}, {ira_num}, {loan_notes}, {partner_paid}, {promissory_note});""")
    con.commit()

def create_loan_instance(amount, stage, app_submitted, funds_wired, start_date, date_paid, loan_num, instance_notes):
    """Create a new loan instance"""
    cur.execute(f"""INSERT INTO loan_instances (amount, stage, app_submitted, funds_wired, start_date, date_paid, loan_num, instance_notes)
                VALUES ({amount}, {stage}, {app_submitted}, {funds_wired}, {start_date}, {date_paid}, {loan_num}, {instance_notes});""")
    con.commit()

def create_client(first_name, last_name, email, phone, address, city, state, zip, client_notes):
    """Create a new client"""
    cur.execute(f"""INSERT INTO clients (first_name, last_name, email, phone, address, city, state, zip, client_notes)
                VALUES ({first_name}, {last_name}, {email}, {phone}, {address}, {city}, {state}, {zip}, {client_notes});""")
    con.commit()

def create_referring_partner(name, email, referral_rate, partner_notes):
    """Create a new referring partner"""
    cur.execute(f"""INSERT INTO referring_partners (name, email, referral_rate, partner_notes)
                VALUES ({name}, {email}, {referral_rate}, {partner_notes});""")
    con.commit()

def create_client_referring_partner(client_id, referring_partner_id):
    """Create a new client referring partner relationship"""
    cur.execute(f"""INSERT INTO client_referring_partner (client_id, referring_partner_id)
                VALUES ({client_id}, {referring_partner_id});""")
    con.commit()

def create_loan_client(loan_id, client_id):
    """Create a new loan client relationship"""
    cur.execute(f"""INSERT INTO loan_client (loan_id, client_id)
                VALUES ({loan_id}, {client_id});""")
    con.commit()


In [8]:
# Functions to create:
# Reporting features
# 1. Create pay period report


# Requirements
"""
Your software must demonstrate the ability to insert, modify, delete, and retrieve (or query) data.
This module requires more than just creating the database and determining SQL commands. You must write software that builds the SQL commands, submits them, receives the results from the database, and uses the results in some way."""
"""Stretch: perform a join in your software between two of the tables.
"""

# TODO
# Insert the airtable data
# Display results in a simple dash app
